In [7]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilution, replenish, dilute_out_species
from scripts.species import DNA, RNA, Protein, Resource
import scripts.models 


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

NameError: name 'molecules_0' is not defined

In [ ]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.1
T7_DNA.lc = 0.1

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 1
GFP_DNA.lc = 1

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.2
Repressor_DNA.lc = 0.2


#resources
R.ic =500
R.lc =500

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):

    molecules_0[i] = molecules_list[i].ic


In [ ]:
#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 1;
HC = 1;
KTL=10;
KTX=1;
Kr = 1
lamb1=0;
lamb2=0;

In [ ]:
#define model
def Self_regenerate_model(molecules,t,alpha,beta,K,Kr,KTL,KTX,lamb1,lamb2):
    T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R = molecules
    TX = 1
    TL = 1
    return np.array(
        [
            0,
            alpha*TX*T7_DNA*T7*R/(R+K)*(Kr/(Repressor+Kr)),
            beta*TL*T7_RNA*(R/(R+K)),
            0,
            alpha*TX*GFP_DNA*T7*R/(R+K),
            beta*TL*GFP_RNA*R/(R+K),
            0,
            alpha*TX*Repressor_DNA*T7*R/(R+K),
            beta*TL*Repressor_RNA*R/(R+K),
            -alpha*TX*T7_DNA*T7*R/(R+K)-alpha*TX*GFP_DNA*T7*R/(R+K)-alpha*TX*Repressor_DNA*T7*R/(R+K)-beta*TL*T7_RNA*R/(R+K)-beta*TL*GFP_RNA*R/(R+K)-beta*TL*Repressor_RNA*R/(R+K),
        ]
    )
    

In [ ]:
#stage 1 kick start
result_all=molecules_0 #initial condition

DR = 0.2


for n in range (1,16):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules_0, t, args=(alpha,beta,K,Kr,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]
    molecules_0[T7.idx] = T7.lc*DR+(1-DR)*molecules_0[T7.idx]

    ###diute out
    molecules_0[T7_RNA.idx] = (1-DR)*molecules_0[T7_RNA.idx] 
    molecules_0[GFP_RNA.idx] = (1-DR)*molecules_0[GFP_RNA.idx]
    molecules_0[GFP.idx] = (1-DR)*molecules_0[GFP.idx]
    molecules_0[Repressor_RNA.idx] = (1-DR)*molecules_0[Repressor_RNA.idx]
    molecules_0[Repressor.idx] = (1-DR)*molecules_0[Repressor.idx]


#stage 2 self regeneration 

for n in range (16,64):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules_0, t, args=(alpha,beta,K,Kr,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]
    

    ###diute out
    molecules_0[T7_RNA.idx] = (1-DR)*molecules_0[T7_RNA.idx] 
    molecules_0[T7.idx] = (1-DR)*molecules_0[T7.idx] 
    molecules_0[GFP_RNA.idx] = (1-DR)*molecules_0[GFP_RNA.idx]
    molecules_0[GFP.idx] = (1-DR)*molecules_0[GFP.idx]
    molecules_0[Repressor_RNA.idx] = (1-DR)*molecules_0[Repressor_RNA.idx]
    molecules_0[Repressor.idx] = (1-DR)*molecules_0[Repressor.idx]

    dilute_out_species(( molecules_0[Repressor_RNA.idx],molecules_0[GFP_RNA.idx]))



for n in range (64,96):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules_0, t, args=(alpha,beta,K,Kr,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]

    ###diute out
    molecules_0[T7_RNA.idx] = dilution(molecules_0[T7_RNA.idx],DR)
    molecules_0[GFP_RNA.idx] = (1-DR)*molecules_0[GFP_RNA.idx]
    molecules_0[T7.idx] = (1-DR)*molecules_0[T7.idx] 
    molecules_0[GFP.idx] = (1-DR)*molecules_0[GFP.idx]
    molecules_0[T7_DNA.idx] = (1-DR)*molecules_0[T7_DNA.idx]
    molecules_0[GFP_DNA.idx] = (1-DR)*molecules_0[GFP_DNA.idx]
    molecules_0[Repressor_DNA.idx] = (1-DR)*molecules_0[Repressor_DNA.idx]
    molecules_0[Repressor_RNA.idx] = (1-DR)*molecules_0[Repressor_RNA.idx]
    molecules_0[Repressor.idx] = (1-DR)*molecules_0[Repressor.idx]



result_all = np.resize(result_all,(96,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()


TypeError: dilute_out_species() missing 1 required positional argument: 'molecules_0'

In [ ]:
dilute_out_species(0)

TypeError: 'int' object is not iterable

In [ ]:
t = np.linspace(0, 1440, 96)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_label="dimensionless concentration",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][3]


# Populate glyphs
p.line(
    t/60, Repressor.result, line_width=2, color=colors[0]
)
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)


show(p)


In [5]:
np.shape(result_all)

NameError: name 'np' is not defined

In [ ]:
dilute_out_species((T7,T7_DNA))

NameError: name 'dilute_out_species' is not defined

In [ ]:
molecules_0

array([5.12000000e-02, 2.24382621e-01, 1.05034905e+00, 1.00000000e+00,
       7.93444398e+01, 2.46333971e+02, 2.00000000e-01, 1.58688880e+01,
       4.92667942e+01, 1.00000000e+02])